In [1]:
import time
from selenium import webdriver

driver = webdriver.Chrome("/Users/donghunbyun/Desktop/workspace/jupyter/bigdata/chromedriver")

In [2]:
from bs4 import BeautifulSoup

In [3]:
driver.get("https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=1&sido=&gugun=&store=")

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
    
stores = soup.select('#contents > div.content > fieldset > fieldset > div.tableType01 > table > tbody > tr')

In [4]:
store_location = stores[0].select('td')[0].text
store_name = stores[0].select('td > a')[0].text
store_address = stores[0].select('td > a')[1].text
store_phone = stores[0].select('td')[-1].text

In [5]:
print(store_location, store_name, store_address, store_phone)

충북 청주시 서원구 청주성화점 충청북도 청주시 서원구 신성화로 39 (성화동) 1~2층 043-233-6296


In [6]:
def hollys_searching(page):
    url = f"https://www.hollys.co.kr/store/korea/korStore2.do?pageNo={page}&sido=&gugun=&store="
    return url

In [7]:
import json
import requests

In [8]:
kakao_API_key = "5e96943c943636b37f24cd5c657d92b3"

In [9]:
store_name = '할리스 덕수궁점'

url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + store_name
headers = {"Authorization": f"KakaoAK {kakao_API_key}"}

result = json.loads(str(requests.get(url, headers=headers).text))

print(float(result['documents'][0]['y']), float(result['documents'][0]['x']))

37.564553295534 126.97655894494


In [10]:
address = '경기도 양주시 부흥로 2239'

url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
headers = {"Authorization": f"KakaoAK {kakao_API_key}"}

result = json.loads(str(requests.get(url, headers=headers).text))

print(len(result['documents']))

1


In [11]:
def getLatLng(store_name, store_address):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + '할리스 ' + store_name
    headers = {"Authorization": f"KakaoAK {kakao_API_key}"}

    result = json.loads(str(requests.get(url, headers=headers).text))
    
    if len(result['documents']) < 1:
        
        address = store_address.split()
        
        
        for i in range(len(address), 0, -1):
            url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + ' '.join(address[:i])

            result = json.loads(str(requests.get(url, headers=headers).text))
            
            if len(result['documents']) > 0:
                break
    
    
    location = result['documents'][0]
    
    return float(location['y']), float(location['x'])

In [12]:
def get_store(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    stores = soup.select('#contents > div.content > fieldset > fieldset > div.tableType01 > table > tbody > tr')
    
    data = []
    
    for store in stores:
        store_location = store.select('td')[0].text
        store_name = store.select('td > a')[0].text
        store_address = store.select('td > a')[1].text
        store_phone = store.select('td')[-1].text
        
        store_lat, store_lng = getLatLng(store_name, store_address)
                
        data.append([store_location, store_name, store_address, store_phone, store_lat, store_lng])
    
    return data

In [13]:
result = []

for i in range(1, 57):
    url = hollys_searching(i)

    driver.get(url)
    
    data = get_store(driver)
    
    result += data

In [14]:
len(result)

560

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame(result)
df.columns = ["지역", "매장명", "주소", "전화번호", '위도', '경도']

In [17]:
df.head()

,지역,매장명,주소,전화번호,위도,경도
0,충북 청주시 서원구,청주성화점,충청북도 청주시 서원구 신성화로 39 (성화동) 1~2층,043-233-6296,36.618438,127.458145
1,경기 안성시,(하)안성맞춤휴게소점,경기도 안성시 서운면 평택제천고속도로 40 (안성맞춤휴게소음성방향) 할리스커피,031-674-4406,36.968091,127.274312
2,경기 안성시,(상)안성맞춤휴게소점,경기도 안성시 서운면 평택제천고속도로 41 (안성맞춤휴게소평택방향) .,031-673-3822,36.970316,127.274578
3,전남 나주시,나주빛가람혁신도시점,전라남도 나주시 배멧3길 5-4 (빛가람동) .,061-334-5466,35.018746,126.785799
4,충남 공주시,(상)정안알밤휴게소점,충청남도 공주시 정안면 논산천안고속도로 58 (정안상행휴게소) 1층,041-858-0562,36.569450,127.115102


In [18]:
df.to_excel('./files/hollys_stores.xlsx')

In [19]:
korea_hollys = pd.read_excel('./files/hollys_stores.xlsx')

In [20]:
import folium

In [25]:
hollys_map = folium.Map(
    location=[36.45, 127.75],
    tiles='Stamen Terrain',
    zoom_start=8
)
hollys_map

In [26]:
for idx in korea_hollys.index:
    lat = korea_hollys.loc[idx, '위도']
    lng = korea_hollys.loc[idx, '경도']

    folium.CircleMarker(
        location=[lat, lng],
        fill = True, 
        fill_color='green', 
        fill_opacity=1,
        color='yellow', 
        weight=1,
        radius=3
    ).add_to(hollys_map)

hollys_map